# ADM Explained

__Pega__

__2023-03-15__

This notebook shows exactly how all the values in an ADM model report
are calculated. It also shows how the propensity is calculated for a
particular customer.

We use one of the shipped datamart exports for the example. This is a
model very similar to one used in some of the ADM PowerPoint/Excel deep
dive examples. You can change this notebook to apply to your own data.



In [ ]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio

pio.renderers.default = "notebook_connected"

import sys
import re

sys.path.append("../../../")
sys.path.append("../../python")
import pandas as pd

pd.set_option("display.max_colwidth", 0)

format_binning_derived = {
    "Positives": "{:.0f}",
    "Negatives": "{:.0f}",
    "Responses %": "{:.2f}",
    "Positives %": "{:.2f}",
    "Negatives %": "{:.2f}",
    "Propensity": "{:.4f}",
}
format_lift = {"Positives": "{:.0f}", "Negatives": "{:.0f}", "Lift": "{:.4f}"}
format_z_ratio = {
    "Positives": "{:.0f}",
    "Negatives": "{:.0f}",
    "Positives %": "{:.2f}",
    "Negatives %": "{:.2f}",
    "ZRatio": "{:.4f}",
}
format_log_odds = {
    "Positives": "{:.0f}",
    "Negatives": "{:.0f}",
    "Positives %": "{:.2f}",
    "LogOdds %": "{:.4f}",
    "ModifiedLogOdds": "{:.4f}",
}
format_classifier = {"Positives": "{:.0f}", "Negatives": "{:.0f}"}

In [15]:
import polars as pl
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from typing import List
from math import log
from great_tables import GT
from pdstools import datasets, cdh_utils

pl.Config.set_fmt_str_lengths(100);

In [3]:
model_name = "AutoNew84Months"
predictor_name = "Customer.NetWealth"
channel = "Web"

For the example we pick one particular model over a channel.
To explain the ADM model report, we use one of the active predictors as an
example. Swap for any other predictor when using different data.

In [7]:
dm = datasets.cdh_sample()

model = dm.combined_data.filter(
    (pl.col("Name") == model_name) & (pl.col("Channel") == channel)
)

modelpredictors = (
    dm.combined_data.join(
        model.select(pl.col("ModelID").unique()), on="ModelID", how="inner"
    )
    .filter(pl.col("EntryType") != "Inactive")
    .with_columns(
        Action=pl.concat_str(["Issue", "Group"], separator="/"),
        PredictorName=pl.col("PredictorName").cast(pl.Utf8),
    )
    .collect()
)

predictorbinning = modelpredictors.filter(
    pl.col("PredictorName") == predictor_name
).sort("BinIndex")

In [13]:
model_id = None

if (modelpredictors.select(pl.col("ModelID").unique()).shape[0] > 1) and (
    model_id is None
):
    display(
        model.group_by("ModelID")
        .agg(
            number_of_predictors=pl.col("PredictorName").n_unique(),
            model_performance=cdh_utils.weighted_performance_polars() * 100,
            response_count=pl.sum("ResponseCount"),
        )
        .collect()
        .to_pandas()
    )
    raise Exception(
        f"**{model_name}** model has multiple instances."
        "\nThis could be due to the same model name being used in different configurations, directions, issues, or having multiple treatments."
        "\nTo ensure the selection of a unique model, please choose a model_id from the table above and update the `model_id` variable at the top of this cell."
        "\nAfterward, rerun this cell."
        f"\nSee model IDs in {model_name} model above:"
    )
if model_id is not None:
    if (
        model_id
        not in modelpredictors.select(pl.col("ModelID").unique())
        .get_column("ModelID")
        .to_list()
    ):
        raise Exception(
            f"The {model_name} model does not have a model ID: {model_id}."
            f"Please ensure that the spelling of the model ID is correct."
            f"You can run `modelpredictors.select(pl.col('ModelID').unique().implode()).row(0)` to see the exact spellings of your IDs."
            "After updating the `model_id`, you can restart the notebook from the beginning."
        )

    predictors_in_selected_model = (
        modelpredictors.filter(pl.col("ModelID") == model_id)
        .select(pl.col("PredictorName").unique())
        .get_column("PredictorName")
        .to_list()
    )
    if predictor_name not in predictors_in_selected_model:
        raise Exception(
            f"{predictor_name} is not a predictor of the model with ID: {model_id}."
            "Please choose one of the available predictors below and update the **predictor_name** variable in the cell above:"
            f"\nAvailable Predictors:\n{predictors_in_selected_model}."
        )

    modelpredictors = modelpredictors.filter(pl.col("ModelID") == model_id)
    predictorbinning = predictorbinning.filter(pl.col("ModelID") == model_id)
    print(f"{model_name} model with **{model_id}** model ID is selected successfully.")

## Model Overview

The selected model is shown below. Only the currently active predictors are used for the propensity calculation, so only showing those.



In [53]:
from great_tables import loc, style

GT(
    modelpredictors.select(
        pl.col("Action").unique(),
        pl.col("Channel").unique(),
        pl.col("Name").unique(),
        pl.col("PredictorName")
        .unique()
        .sort()
        .implode()
        .list.join(", ")
        .alias("Active Predictors"),
        (pl.col("Performance").unique() * 100).alias("Model Performance (AUC)"),
    ).transpose(include_header=True)
).tab_header("Overview").tab_options(column_labels_hidden=True).tab_style(
    style=style.text(weight="bold"), locations=loc.body(columns="column")
)

GT(_tbl_data=shape: (5, 2)
┌─────────────────────────┬──────────────────────────────────────────────────────┐
│ column                  ┆ column_0                                             │
│ ---                     ┆ ---                                                  │
│ str                     ┆ str                                                  │
╞═════════════════════════╪══════════════════════════════════════════════════════╡
│ Action                  ┆ Sales/AutoLoans                                      │
│ Channel                 ┆ Web                                                  │
│ Name                    ┆ AutoNew84Months                                      │
│ Active Predictors       ┆ Classifier, Customer.Age, Customer.AnnualIncome,     │
│                         ┆ Customer.BusinessSegment, Customer.CLV, Customer.CL… │
│ Model Performance (AUC) ┆ 77.4901                                              │
└─────────────────────────┴──────────────────────────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x16a27a7b0>, _boxhead=Boxhead([ColInfo(var='column', type=<ColInfoTypeEnum.default: 1>, column_label='column', column_align='left', column_width=None), ColInfo(var='column_0', type=<ColInfoTypeEnum.default: 1>, column_label='column_0', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x16a26f0b0>, _spanners=Spanners([]), _heading=Heading(title='Overview', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=2, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=3, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=4, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)])], _locale=<great_tables._gt_data.Locale object at 0x169f17e90>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), 

## Binning of the selected Predictor

The Model Report in Prediction Studio for this model will have a predictor binning plot like below.

All numbers can be derived from just the number of positives and negatives in each bin that are stored in the ADM Data Mart. The next sections will show exactly how that is done.

In [74]:
display(
    GT(
        (
            predictorbinning.group_by("PredictorName")
            .agg(
                pl.first("ResponseCount").cast(pl.Utf8).alias("# Responses"),
                pl.n_unique("BinIndex").cast(pl.Utf8).alias("# Bins"),
                (pl.first("PerformanceBin") * 100)
                .cast(pl.Utf8)
                .alias("Predictor Performance(AUC)"),
            )
            .rename({"PredictorName": "Predictor Name"})
            .transpose(include_header=True)
        )
    )
    .tab_header("Predictor information")
    .tab_options(column_labels_hidden=True)
    .tab_style(style=style.text(weight="bold"), locations=loc.body(columns="column"))
    .tab_options(table_margin_left=0)
)

fig = dm.plot.predictor_binning(
    model_id=modelpredictors.get_column("ModelID").unique().to_list()[0],
    predictor_name=predictor_name,
)
fig.update_layout(width=600, height=400)


GT(_tbl_data=shape: (4, 2)
┌────────────────────────────┬────────────────────┐
│ column                     ┆ column_0           │
│ ---                        ┆ ---                │
│ str                        ┆ str                │
╞════════════════════════════╪════════════════════╡
│ Predictor Name             ┆ Customer.NetWealth │
│ # Responses                ┆ 1636               │
│ # Bins                     ┆ 8                  │
│ Predictor Performance(AUC) ┆ 72.2077            │
└────────────────────────────┴────────────────────┘, _body=<great_tables._gt_data.Body object at 0x16b91b4a0>, _boxhead=Boxhead([ColInfo(var='column', type=<ColInfoTypeEnum.default: 1>, column_label='column', column_align='left', column_width=None), ColInfo(var='column_0', type=<ColInfoTypeEnum.default: 1>, column_label='column_0', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x16b91fe60>, _spanners=Spanners([]), _heading=Heading(title='Predictor information', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=2, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='column', rownum=3, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None)])], _locale=<great_tables._gt_data.Locale object at 0x16b91ecc0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value=0), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A

In [ ]:
BinPositives = pl.col("BinPositives")
BinNegatives = pl.col("BinNegatives")
sumPositives = pl.sum("BinPositives")
sumNegatives = pl.sum("BinNegatives")

# TODO: add totals for first 5 columns, base rate for 6 and 0,1, see R version

binstats = predictorbinning.select(
    pl.col("BinSymbol").alias("Range/Symbol"),
    ((BinPositives + BinNegatives) / (sumPositives + sumNegatives))
    .round(3)
    .alias("Responses (%)"),
    BinPositives.alias("Positives"),
    (BinPositives / sumPositives).round(3).alias("Positives (%)"),
    BinNegatives.alias("Negatives"),
    (BinNegatives / sumNegatives).round(3).alias("Negatives (%)"),
    (BinPositives / (BinPositives + BinNegatives)).round(4).alias("Propensity (%)"),
    cdh_utils.zRatio(negCol=BinNegatives, posCol=BinPositives),
    (
        (BinPositives / (BinPositives + BinNegatives))
        / (sumPositives / (BinPositives + BinNegatives).sum())
    ).alias("Lift"),
)

binstats.vstack(
    pl.DataFrame(
        dict(
            zip(
                binstats.columns,
                ["Total"]
                + [
                    binstats.select(pl.sum(col)).row(0)[0]
                    for col in binstats.columns[1:]
                ],
            )
        ),
        schema=binstats.schema,
    )
).to_pandas().set_index("Range/Symbol")


## Bin Statistics

### Positive and Negative ratios

Internally, ADM only keeps track of the total counts of positive and negative responses in each bin. Everything else is derived from those numbers. The percentages and totals are trivially derived, and the propensity is just the number of positives divided by the total. The numbers calculated here match the numbers from the datamart table exactly.

In [ ]:
binningDerived = predictorbinning.select(
    pl.col("BinSymbol").alias("Range/Symbol"),
    BinPositives.alias("Positives"),
    BinNegatives.alias("Negatives"),
    (((BinPositives + BinNegatives) / (sumPositives + sumNegatives)) * 100)
    .round(2)
    .alias("Responses %"),
    ((BinPositives / sumPositives) * 100).round(2).alias("Positives %"),
    ((BinNegatives / sumNegatives) * 100).round(2).alias("Negatives %"),
    (BinPositives / (BinPositives + BinNegatives)).round(4).alias("Propensity"),
)
binningDerived.to_pandas(use_pyarrow_extension_array=True).set_index(
    "Range/Symbol"
).style.format(format_binning_derived).set_properties(
    color="#0000FF", subset=["Responses %", "Positives %", "Negatives %", "Propensity"]
)

### Lift

Lift is the ratio of the propensity in a particular bin over the average propensity. So a value of 1 is the average, larger than 1 means higher propensity, smaller means lower propensity:

In [ ]:
Positives = pl.col("Positives")
Negatives = pl.col("Negatives")
sumPositives = pl.sum("Positives")
sumNegatives = pl.sum("Negatives")
binningDerived.select(
    "Range/Symbol",
    "Positives",
    "Negatives",
    (
        (Positives / (Positives + Negatives))
        / (sumPositives / (Positives + Negatives).sum())
    ).alias("Lift"),
).to_pandas().set_index("Range/Symbol").style.format(format_lift).set_properties(
    **{"color": "blue"}, subset=["Lift"]
)

### Z-Ratio

The Z-Ratio is also a measure of the how the propensity in a bin differs from the average, but takes into account the size of the bin and thus is statistically more relevant. It represents the number of standard deviations from the average, so centers around 0. The wider the spread, the better the predictor is.
$$\frac{posFraction-negFraction}{\sqrt(\frac{posFraction*(1-posFraction)}{\sum positives}+\frac{negFraction*(1-negFraction)}{\sum negatives})}$$ 

See the calculation here, which is also included in [cdh_utils' zRatio()](https://pegasystems.github.io/pega-datascientist-tools/Python/autoapi/pdstools/utils/cdh_utils/index.html#pdstools.utils.cdh_utils.zRatio).

In [ ]:
def zRatio(
    posCol: pl.Expr = pl.col("BinPositives"), negCol: pl.Expr = pl.col("BinNegatives")
) -> pl.Expr:
    def getFracs(posCol=pl.col("BinPositives"), negCol=pl.col("BinNegatives")):
        return posCol / posCol.sum(), negCol / negCol.sum()

    def zRatioimpl(
        posFractionCol=pl.col("posFraction"),
        negFractionCol=pl.col("negFraction"),
        PositivesCol=pl.sum("BinPositives"),
        NegativesCol=pl.sum("BinNegatives"),
    ):
        return (
            (posFractionCol - negFractionCol)
            / (
                (posFractionCol * (1 - posFractionCol) / PositivesCol)
                + (negFractionCol * (1 - negFractionCol) / NegativesCol)
            ).sqrt()
        ).alias("ZRatio")

    return zRatioimpl(*getFracs(posCol, negCol), posCol.sum(), negCol.sum())


binningDerived.select(
    "Range/Symbol", "Positives", "Negatives", "Positives %", "Negatives %"
).with_columns(zRatio(Positives, Negatives)).to_pandas().set_index(
    "Range/Symbol"
).style.format(format_z_ratio).set_properties(**{"color": "blue"}, subset=["ZRatio"])

## Predictor AUC


The predictor AUC is the univariate performance of this predictor against the outcome. This too can be derived from the positives and negatives and
there is  a convenient function in pdstools to calculate it directly from the positives and negatives.

This function is implemented in cdh_utils: [cdh_utils.auc_from_bincounts()](https://pegasystems.github.io/pega-datascientist-tools/Python/autoapi/pdstools/utils/cdh_utils/index.html#pdstools.utils.cdh_utils.auc_from_bincounts).

In [ ]:
pos = binningDerived.get_column("Positives").to_numpy()
neg = binningDerived.get_column("Negatives").to_numpy()
probs = binningDerived.get_column("Propensity").to_numpy()
order = np.argsort(probs)

FPR = np.cumsum(neg[order]) / np.sum(neg[order])
TPR = np.cumsum(pos[order]) / np.sum(pos[order])
TPR = np.insert(TPR, 0, 0, axis=0)
FPR = np.insert(FPR, 0, 0, axis=0)
# Checking whether classifier labels are correct
if TPR[1] < 1 - FPR[1]:
    temp = FPR
    FPR = TPR
    TPR = temp
auc = cdh_utils.auc_from_bincounts(pos=pos, neg=neg, probs=probs)

fig = px.line(
    x=[1 - x for x in FPR],
    y=TPR,
    labels=dict(x="Specificity", y="Sensitivity"),
    title=f"AUC = {auc.round(3)}",
    width=700,
    height=700,
    range_x=[1, 0],
    template="none",
)
fig.add_shape(type="line", line=dict(dash="dash"), x0=1, x1=0, y0=0, y1=1)
fig.show()


## Naive Bayes and Log Odds

The basis for the Naive Bayes algorithm is Bayes' Theorem:

$$p(C_k|x) = \frac{p(x|C_k)*p(C_k)}{p(x)}$$

with $C_k$ the outcome and $x$ the customer. Bayes' theorem turns the
question "what's the probability to accept this action given a customer" around to 
"what's the probability of this customer given an action". With the independence
assumption, and after applying a log odds transformation we get a log odds score 
that can be calculated efficiently and in a numerically stable manner:

$$log\ odds\ score = \sum_{p\ \in\ active\ predictors}log(p(x_p|Positive)) + log(p_{positive}) - \sum_plog(p(x_p|Negative)) - log(p_{negative})$$
note that the _prior_ can be written as:

$$log(p_{positive}) - log(p_{negative}) = log(\frac{TotalPositives}{Total})-log(\frac{TotalNegatives}{Total}) = log(TotalPositives) - log(TotalNegatives)$$


## Predictor Contribution

The contribution (_conditional log odds_) of an active predictor $p$ for bin $i$ with the number
of positive and negative responses in $Positives_i$ and $Negatives_i$ is calculated as (note the "laplace smoothing" to avoid log 0 issues):

$$contribution_p = \log(Positives_i+\frac{1}{nBins}) - \log(Negatives_i+\frac{1}{nBins}) - \log(1+\sum_{i\ = 1..nBins}{Positives_i}) + \log(1+\sum_i{Negatives_i})$$


In [ ]:
N = binningDerived.shape[0]
binningDerived.with_columns(
    LogOdds=(pl.col("Positives %") / pl.col("Negatives %")).log(),
    ModifiedLogOdds=(
        ((Positives + 1 / N).log() - (Positives.sum() + 1).log())
        - ((Negatives + 1 / N).log() - (Negatives.sum() + 1).log())
    ),
).drop("Responses %", "Propensity").to_pandas().set_index("Range/Symbol").style.format(
    format_log_odds
).set_properties(**{"color": "blue"}, subset=["LogOdds", "ModifiedLogOdds"])

## Propensity mapping

### Log odds contribution for all the predictors

The final score is loosely referred to as "the average contribution" but
in fact is a little more nuanced. The final score is calculated as:

$$score = \frac{\log(1 + TotalPositives) – \log(1 + TotalNegatives) + \sum_p contribution_p}{1 + nActivePredictors}$$

Here, $TotalPositives$ and $TotalNegatives$ are the total number of
positive and negative responses to the model.

Below an example. From all the active predictors of the model 
we pick a value (in the middle for numerics, first symbol
for symbolics) and show the (modified) log odds. The final score is
calculated per the above formula, and this is the value that is mapped
to a propensity value by the classifier (which is constructed using the
[PAV(A)](https://en.wikipedia.org/wiki/Isotonic_regression) algorithm).


In [ ]:
def middleBin():
    return pl.col("BinIndex") == (pl.max("BinIndex") / 2).floor().cast(pl.UInt32)


if not all(
    col in modelpredictors.columns for col in ["BinLowerBound", "BinUpperBound"]
):

    def extract_numbers_in_contents(s: str, index):
        numbers = re.findall(r"[-+]?\d*\.\d+|\d+", s)
        try:
            number = float(numbers[index])
        except:
            number = 0
        return number

    modelpredictors = modelpredictors.with_columns(
        pl.col("Contents").cast(pl.Utf8)
    ).with_columns(
        pl.when(pl.col("Type") == "numeric")
        .then(
            pl.col("Contents").map_elements(
                lambda col: extract_numbers_in_contents(col, 0)
            )
        )
        .otherwise(pl.lit(-9999))
        .alias("BinLowerBound")
        .cast(pl.Float32),
        pl.when(pl.col("Type") == "numeric")
        .then(
            pl.col("Contents").map_elements(
                lambda col: extract_numbers_in_contents(col, 1)
            )
        )
        .otherwise(pl.lit(-9999))
        .alias("BinUpperBound")
        .cast(pl.Float32),
    )


def RowWiseLogOdds(Bin, Positives, Negatives):
    Bin, N = Bin.list.get(0) - 1, Positives.list.len()
    Pos, Neg = Positives.list.get(Bin), Negatives.list.get(Bin)
    PosSum, NegSum = Positives.list.sum(), Negatives.list.sum()
    return (
        ((Pos + (1 / N)).log() - (PosSum + 1).log())
        - (((Neg + (1 / N)).log()) - (NegSum + 1).log())
    ).alias("Modified Log odds")


df = (
    modelpredictors.filter(pl.col("PredictorName") != "Classifier")
    .group_by("PredictorName")
    .agg(
        Value=pl.when(pl.col("Type").first() == "numeric")
        .then(
            ((pl.col("BinLowerBound") + pl.col("BinUpperBound")) / 2).where(middleBin())
        )
        .otherwise(pl.col("BinSymbol").str.split(",").list.first().where(middleBin())),
        Bin=pl.col("BinIndex").where(middleBin()),
        Positives=pl.col("BinPositives"),
        Negatives=pl.col("BinNegatives"),
    )
    .with_columns(
        pl.col(["Positives", "Negatives"]).list.get(pl.col("Bin").list.get(0) - 1),
        pl.col("Bin", "Value").list.get(0),
        LogOdds=RowWiseLogOdds(pl.col("Bin"), pl.col("Positives"), pl.col("Negatives")),
    )
    .sort("PredictorName")
)

classifier = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .with_columns(
        Propensity=(BinPositives / (BinPositives / BinNegatives)),
        AdjustedPropensity=((0.5 + BinPositives) / (1 + BinPositives + BinNegatives)),
        ZRatio=cdh_utils.zRatio(negCol=BinNegatives, posCol=BinPositives),
        Lift=(
            (BinPositives / (BinPositives + BinNegatives))
            / (sumPositives / (BinPositives + BinNegatives).sum())
        ),
    )
    .select(
        [
            pl.col("BinIndex").alias("Index"),
            pl.col("BinSymbol").alias("Bin"),
            BinPositives.alias("Positives"),
            BinNegatives.alias("Negatives"),
            ((pl.cum_sum("BinResponseCount") / pl.sum("BinResponseCount")) * 100).alias(
                "Cum. Total (%)"
            ),
            (pl.col("BinPropensity") * 100).alias("Propensity (%)"),
            (pl.col("AdjustedPropensity") * 100).alias("Adjusted Propensity (%)"),
            ((pl.cum_sum("BinPositives") / pl.sum("BinPositives")) * 100).alias(
                "Cum Positives (%)"
            ),
            pl.col("ZRatio"),
            (pl.col("Lift") * 100).alias("Lift(%)"),
            pl.col("BinResponseCount").alias("Responses"),
        ]
    )
)
classifierLogOffset = log(1 + classifier["Positives"].sum()) - log(
    1 + classifier["Negatives"].sum()
)

propensity_mapping = (
    df.vstack(
        pl.DataFrame(
            dict(
                zip(
                    df.columns,
                    ["Final Score"]
                    + [None] * 4
                    + [(df["LogOdds"].sum() + classifierLogOffset) / (len(df) + 1)],
                )
            ),
            schema=df.schema,
        )
    )
    .to_pandas()
    .set_index("PredictorName")
    .style.set_properties(**{"color": "blue"}, subset=["LogOdds"])
)

propensity_mapping

## Classifier

The success rate is defined as $\frac{positives}{positives+negatives}$ per bin. 

The adjusted propensity that is returned is a small modification (Laplace smoothing) to this and calculated as $\frac{0.5+positives}{1+positives+negatives}$ so empty models return a propensity of 0.5.


In [ ]:
# TODO see if we can port the "getActiveRanges" code to python so to highlight the classifier rows that are "active"

classifier.drop("Responses").to_pandas().set_index("Index").style.format(
    format_classifier
).set_properties(**{"color": "blue"}, subset=["Adjusted Propensity (%)"])

## Final Propensity

Below the classifier mapping. On the x-axis the binned scores (log odds values), on the y-axis the Propensity. Note the returned propensities are following a slightly adjusted formula, see the table above. The bin that contains the calculated final score is highlighted.

In [ ]:
score = propensity_mapping.data.loc["Final Score", "LogOdds"]
score_bin = modelpredictors.filter(pl.col("EntryType") == "Classifier").select(
    pl.col("BinSymbol").where(
        pl.lit(score).is_between(pl.col("BinLowerBound"), pl.col("BinUpperBound"))
    )
)["BinSymbol"][0]
score_responses = modelpredictors.filter(
    (pl.col("EntryType") == "Classifier") & (pl.col("BinSymbol") == score_bin)
)["BinResponseCount"][0]
score_bin_index = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")["BinSymbol"]
    .to_list()
    .index(score_bin)
)
score_propensity = classifier.to_pandas().iloc[score_bin_index][
    "Adjusted Propensity (%)"
]

adjusted_propensity = (
    modelpredictors.filter(pl.col("EntryType") == "Classifier")
    .with_columns(
        AdjustedPropensity=((0.5 + BinPositives) / (1 + BinPositives + BinNegatives)),
    )
    .select(
        pl.col("AdjustedPropensity").where(
            (pl.col("BinLowerBound") < score) & (pl.col("BinUpperBound") > score)
        )
    )["AdjustedPropensity"][0]
    * 100
)
adjusted_propensity = round(adjusted_propensity, 2)

fig = ADMVisualisations.distribution_graph(
    modelpredictors.filter(pl.col("EntryType") == "Classifier"),
    "Propensity distribution",
).add_annotation(
    x=score_bin,
    y=score_propensity / 100,
    text=f"Returned propensity: {score_propensity:.2f}%",
    bgcolor="#FFFFFF",
    bordercolor="#000000",
    showarrow=False,
    yref="y2",
    opacity=0.7,
)
bin_index = list(fig.data[0]["x"]).index(score_bin)
fig.data[0]["marker_color"] = (
    ["grey"] * bin_index
    + ["#1f77b4"]
    + ["grey"] * (classifier.shape[0] - bin_index - 1)
)
fig